# yolov3 module

In [1]:
import numpy as np
import sys

# tvm
import tvm
from tvm import te
from tvm import relay

from tvm.relay.testing.darknet import __darknetffi__

## Converting module

In [2]:
# cfg path
cfg_path = "../yolov3/yolov3.cfg"

# weights path
weights_path = "../yolov3/yolov3.weights"

# lib path
lib_path = "../yolov3/libdarknet2.0.so"

In [3]:
DARKNET_LIB = __darknetffi__.dlopen(lib_path)
print(DARKNET_LIB)

<cffi.api._make_ffi_library.<locals>.FFILibrary object at 0x7fe8049a2048>


In [7]:
net = DARKNET_LIB.load_network(cfg_path.encode("utf-8"), weights_path.encode("utf-8"), 0)
print(net)
dtype = "float32"
batch_size = 1

data = np.empty([batch_size, net.c, net.h, net.w], dtype)
shape_dict = {"data": data.shape}
print(shape_dict)
print(net.layers[net.n - 1].classes)

<cdata 'network *' 0xb405de0>
{'data': (1, 3, 416, 416)}
80


In [5]:
print("Converting darknet to relay function...")

Converting darknet to relay function...


In [6]:
mod, params = relay.frontend.from_darknet(net, dtype=dtype, shape=data.shape)

## Compiling the model

In [7]:
target = tvm.target.Target("llvm")
target_host = tvm.target.Target("llvm")

In [8]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, target_host=target_host, params=params, mod_name="yolov3")
print(lib)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


## export module

In [9]:
# output lib path
export_lib_path = "../module/x64/yolov3.so"

# export params path
export_params_path = "../module/x64/yolov3.params"

In [10]:
lib.export_library(export_lib_path)
#tvm.contrib.cc.cross_compiler("aarch64-linux-gnu-g++")

In [11]:
with open(export_params_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))

## Execute on TVM Runtime